In [332]:
lines = readlines("example.txt") .|> l -> split(l, "|") .|> split

10-element Vector{Vector{Vector{SubString{String}}}}:
 [["be", "cfbegad", "cbdgef", "fgaecd", "cgeb", "fdcge", "agebfd", "fecdb", "fabcd", "edb"], ["fdgacbe", "cefdb", "cefbgd", "gcbe"]]
 [["edbfga", "begcd", "cbg", "gc", "gcadebf", "fbgde", "acbgfd", "abcde", "gfcbed", "gfec"], ["fcgedb", "cgb", "dgebacf", "gc"]]
 [["fgaebd", "cg", "bdaec", "gdafb", "agbcfd", "gdcbef", "bgcad", "gfac", "gcb", "cdgabef"], ["cg", "cg", "fdcagb", "cbg"]]
 [["fbegcd", "cbd", "adcefb", "dageb", "afcb", "bc", "aefdc", "ecdab", "fgdeca", "fcdbega"], ["efabcd", "cedba", "gadfec", "cb"]]
 [["aecbfdg", "fbg", "gf", "bafeg", "dbefa", "fcge", "gcbea", "fcaegb", "dgceab", "fcbdga"], ["gecf", "egdcabf", "bgf", "bfgea"]]
 [["fgeab", "ca", "afcebg", "bdacfeg", "cfaedg", "gcfdb", "baec", "bfadeg", "bafgc", "acf"], ["gebdcfa", "ecba", "ca", "fadegcb"]]
 [["dbcfg", "fgd", "bdegcaf", "fgec", "aegbdf", "ecdfab", "fbedc", "dacgb", "gdcebf", "gf"], ["cefg", "dcbef", "fcge", "gbcadfe"]]
 [["bdfegc", "cbegaf", "gecbf", "dfcag

In [333]:
function part1(lines)
    results = [l[2] for l in lines]
    lengths_1_4_7_8 = [2, 4, 3, 7]
    numbers_1_4_7_8 = map(line -> filter(x -> length(x) in lengths_1_4_7_8, line), results)
    return sum(length.(numbers_1_4_7_8))
end

part1 (generic function with 2 methods)

In [334]:
part1(lines)

26

In [372]:
function deduce_c_and_f(input)
    ones = filter(x -> length(x) == 2, input)
    @assert length(ones) == 1
    one_letters = Set(ones[1])
    @assert length(one_letters) == 2
    return one_letters
end

function deduce_a(input, c_and_f)
    sevens = filter(x -> length(x) == 3, input)
    @assert length(sevens) == 1
    a_set = setdiff(Set(sevens[1]), c_and_f)
    @assert length(a_set) == 1
    return a_set
end

function deduce_b_and_d(input, c_and_f)
    fours = filter(x -> length(x) == 4, input)
    @assert length(fours) == 1
    four_letters = fours[1]
    missing_letters = filter(x -> x ∉ c_and_f, four_letters)
    @assert length(missing_letters) == 2
    return Set(missing_letters)
end

function deduce_a_d_and_g(input)
    five_segments_numbers = filter(x -> length(x) == 5, input)
    candidate_letters = union(Set(five_segments_numbers)...)
    candidate_letters = filter(x -> all(x .∈ five_segments_numbers), candidate_letters)
    @assert length(candidate_letters) == 3
    return candidate_letters
end

function disambiguate_c_and_f(input, c_and_f)
    six_segments_numbers = filter(x -> length(x) == 6, input)
    all_letters = union(Set(six_segments_numbers)...)
    everywhere_letters = filter(x -> all(x .∈ six_segments_numbers), all_letters)
    f_set = filter(x -> x ∈ everywhere_letters, c_and_f)
    @assert length(f_set) == 1
    c_set = setdiff(c_and_f, f_set)
    @assert length(c_set) == 1
    return c_set, f_set
end

function deduce_mapping(input)
    c_and_f = deduce_c_and_f(input)
    a_set = deduce_a(input, c_and_f)
    mapping = Dict(first(a_set) => 'a')
    b_and_d = deduce_b_and_d(input, c_and_f)
    a_d_and_g = deduce_a_d_and_g(input)
    d_and_g = setdiff(a_d_and_g, a_set)
    @assert length(d_and_g) == 2
    d_set = intersect(b_and_d, d_and_g)
    @assert length(d_set) == 1
    mapping[first(d_set)] = 'd'
    b_set = setdiff(b_and_d, d_set)
    @assert length(b_set) == 1
    mapping[first(b_set)] = 'b'
    g_set = setdiff(d_and_g, d_set)
    @assert length(g_set) == 1
    mapping[first(g_set)] = 'g'
    c_set, f_set = disambiguate_c_and_f(input, c_and_f)
    mapping[first(c_set)] = 'c'
    mapping[first(f_set)] = 'f'
    remaining_keys = filter(x -> x ∉ keys(mapping), "abcdefg")
    @assert length(remaining_keys) == 1
    remaining_values = filter(x -> x ∉ values(mapping), "abcdefg")
    @assert length(remaining_values) == 1
    mapping[remaining_keys[1]] = remaining_values[1]
    return mapping
end

deduce_mapping (generic function with 1 method)

In [379]:
int_segments = Dict(
    Set(['a', 'b', 'c', 'e', 'f', 'g']) => 0,
    Set(['c', 'f']) => 1 ,
    Set(['a', 'c', 'd', 'e', 'g']) => 2,
    Set(['a', 'c', 'd', 'f', 'g']) => 3,
    Set(['b', 'c', 'd', 'f']) => 4,
    Set(['a', 'b', 'd', 'f', 'g']) => 5,
    Set(['a', 'b', 'd', 'e', 'f', 'g']) => 6,
    Set(['a', 'c', 'f']) => 7,
    Set(['a', 'b', 'c', 'd', 'e', 'f', 'g']) => 8,
    Set(['a', 'b', 'c', 'd', 'f', 'g']) => 9
)

function get_digit(code, mapping)
    decoded = Set([mapping[l] for l in code])
    return int_segments[decoded]
end

function deduce_output_value(line)
    input, output = line[1], line[2]
    mapping = deduce_mapping(input)
    digits = map(x -> get_digit(x, mapping), output)
    return sum([digits[i] * 10^(length(digits) - i) for i in 1:length(digits)])
end

part2(lines) = sum(deduce_output_value.(lines))

part2 (generic function with 1 method)

In [380]:
part2(lines)

61229